In [1]:
from tkinter import *
import tkinter.ttk as ttk

import face_recognition
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def get_cropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)
    
    a,b,c,d= face_locations[0]
    cropped_face = image[a:c,d:b,:]
    return cropped_face

def get_face_embedding(face):
    return face_recognition.face_encodings(face)

def get_face_embedding_dict(dir_path):
    file_list = os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        image_file = os.path.join(dir_path, file)
        face = face_recognition.load_image_file(image_file) ##고친부분1
        embedding = get_face_embedding(face)
        if len(embedding) > 0:  # 얼굴영역 face가 제대로 detect되지 않으면  len(embedding)==0인 경우가 발생하므로 
                    # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
            embedding_dict[os.path.splitext(file)[0]] = embedding[0]
            
    return embedding_dict

def get_face_embedding_dict_save(dir_path,path_to_save):
    file_list = os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        image_file = os.path.join(dir_path, file)
        face = get_cropped_face(image_file)
        
        pillow_image = Image.fromarray(face)
        embedding = get_face_embedding(face)
        if len(embedding) > 0:  # 얼굴영역 face가 제대로 detect되지 않으면  len(embedding)==0인 경우가 발생하므로 
                    # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
            embedding_dict[os.path.splitext(file)[0]] = embedding[0]
            pillow_image.save(path_to_save + os.path.splitext(file)[0] + '.jpg')
    
    return embedding_dict

def get_distance_ch_celeb(name1, name2):
    return np.linalg.norm(embedding_dict_chahghee[name1]-embedding_dict_celeb[name2], ord=2)

def get_sort_key_func_ch_celeb(name1):
    def get_distance_ch_celeb_from_name1(name2):
        return get_distance_ch_celeb(name1, name2)
    return get_distance_ch_celeb_from_name1

def get_nearest_face_ch_celeb(name, top=5):
    sort_key_func = get_sort_key_func_ch_celeb(name)
    sorted_faces = sorted(embedding_dict_celeb.items(), key=lambda x:sort_key_func(x[0]))

    for i in range(top):
        if i == 0 :   # 첫번째로 나오는 이름은 자기 자신일 것이므로 제외합시다. 
            continue
        if sorted_faces[i]:
            print('순위 {} : 이름({}), 거리({})'.format(i, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))
            

In [2]:
dir_path_celeb = os.getenv('HOME')+'/aiffel/face_embedding/images/celebritycrop'
#file_list_celeb = os.listdir(dir_path_celeb)
embedding_dict_celeb = get_face_embedding_dict(dir_path_celeb)

In [5]:
from tkinter import *
import tkinter.ttk as ttk
from googletrans import Translator
from tkinter import filedialog
import tkinter as tk
import PIL.Image
from tkinter.filedialog import askopenfilename
import py_compile
import face_recognition

def get_cropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)
    
    a,b,c,d= face_locations[0]
    cropped_face = image[a:c,d:b,:]
    return cropped_face

def get_face_embedding(face):
    return face_recognition.face_encodings(face)

def get_face_embedding_dict(image):
    face =get_cropped_face(image)
    embedding_dict = {}
    embedding = get_face_embedding(face)
    if len(embedding) > 0:  # 얼굴영역 face가 제대로 detect되지 않으면  len(embedding)==0인 경우가 발생하므로 
                    # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
        embedding_dict['user'] = embedding[0]
            
    return embedding_dict

def get_distance_user_celeb(name1, name2):
    return np.linalg.norm(embedding_dict_user[name1]-embedding_dict_celeb[name2], ord=2)

def get_sort_key_func_user_celeb(name1):
    def get_distance_user_celeb_from_name1(name2):
        return get_distance_user_celeb(name1, name2)
    return get_distance_user_celeb_from_name1

def get_nearest_face_uesr_celeb(name, top=5):
    sort_key_func = get_sort_key_func_user_celeb(name)
    sorted_faces = sorted(embedding_dict_celeb.items(), key=lambda x:sort_key_func(x[0]))

    return sorted_faces        
            
   
def click_find():
    similar_celeb = get_nearest_face_uesr_celeb('user')
    text1_string = ""
    for i in range(1,5):
        if similar_celeb[i]:
            temp_string = '순위 {} : {} \n'.format(i, similar_celeb[i][0])
            text1_string += temp_string
                               
    text1.delete(1.0, "end") 
    text1.insert("end",text1_string)
    
    
def click_open_img():
    global photo
    global embedding_dict_user
    filename = filedialog.askopenfilename(initialdir = "E:/Images", filetypes = ())
    photo = tk.PhotoImage(file=filename)
    photo = photo.subsample(2,2)
    canvas.create_image(200,200, image=photo)
    embedding_dict_user = get_face_embedding_dict(filename)
    
    
print('코드 실행 중..🛸')
window = Tk()
window.title("닮은꼴 찾아주기")

window.geometry("1024x600")

label1 = Label(window, text="사진을 입력하세요") 
label2 = Label(window, text="닮은꼴 연예인은?")
label1.grid(row=0, column=0, padx=20, pady=10)
label2.grid(row=0, column=2, padx=20, pady=10)



text1 = Text(window, width=60, height=40)
#text1.grid(row=2, column=0, padx=20, pady=10)
text1.grid(row=2, column=2, padx=20, pady=10)


canvas = tk.Canvas(window, width=400, height=450)
canvas.grid(row=2, column=0,padx=20, pady=10)
    
action_find = ttk.Button(window, text="찾기", command=click_find)
action_load_image = ttk.Button(window, text = 'Load Image', command = click_open_img)
action_find.grid(row=2, column=1)
action_load_image.grid(row=1, column = 0)

window.mainloop()
print('완료!')

코드 실행 중..🛸
완료!
